# Visualization of Coincident ICESat-2, Sentinel, and Argo Float Data

ICESat-2 hackweek final project 
June 16, 2020  
Nicole Abib

In [ ]:
import os
from icepyx import icesat2data as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py 
import sys  
sys.path.insert(0, '/home/jovyan/leading_to_phytoplankton/scripts')
import readers as rd
import cartopy.crs as ccrs
from astropy.time import Time
import warnings
warnings.filterwarnings('ignore')
# magic function to enable interactive plotting
%matplotlib widget 

In [ ]:
# Define path to sample data
data_loc='/home/jovyan/shared/leading_to_phytoplankton/'
argo_fn = data_loc+'floats_highlats.csv'
# s2_fn = 

In [ ]:
# Quick check of csv file contents
!head $argo_fn

In [ ]:
# Load the csv file with Pandas
argo_df = pd.read_csv(argo_fn)
# Add column names defined in the metadata
argo_df.columns = ['lat', 'lon', 'year', 'month', 'day', 'hour', 
                   'minute', 'Depth (m)', 'bbp (700 nm)', 'temperature', 
                   'salinity', 'chlorophyll']
# Inspect the dataframe
argo_df

In [ ]:
# Check data types
argo_df.info()

In [ ]:
# Check that column labels are correct
argo_df.columns

In [ ]:
# Check head
argo_df.head()

In [ ]:
# Check tail
argo_df.tail()

In [ ]:
argo_df_nan = argo_df.replace(99999, np.nan)
argo_df = argo_df_nan

In [ ]:
# Create a scatter plot showing data locations
ax = argo_df_nan.plot(x='lon', y='lat', kind='scatter', c='k', s=1)
ax.set_title('Location of Argo Floats');

# Load in ICESat-2 Data

In [ ]:
# temp_dloc = '/home/jovyan/tutorial-data/sea-ice/'
temp_dloc = '/home/jovyan/'
# is2_fn = 'ATL03_20190206123200_06120204_002_01.h5'
is2_fn = 'ATL03_20190812180506_06970410_003_01.h5'
is2_f = h5py.File(temp_dloc+is2_fn, 'r')
print(is2_f['orbit_info/sc_orient'][0]) # Check forward/backward orientation

In [ ]:
# Read in icesat 2 data as a pandas dataframe
beam='gt1l'
df03=rd.getATL03(is2_f,beam)
df03.head()

In [ ]:
var= 'heights' #choose which variable we want to plot

## we will want to set colorbar parameters based on the chosen variable
vmin=-10
vmax=30
ticks=np.arange(-20,100,5)

# cartopy - need to make figure and set projection first before adding data
plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.SouthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, -60, -90], ccrs.PlateCarree())
plt.scatter(argo_df['lon'], argo_df['lat'], c= 'k',s=2,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both');

# When i have coincident data, I can plot these on top of eachother

In [ ]:
df03.head()

In [ ]:
## create bounding box of interest:
lonmin,lonmax,latmin,latmax= [-90,-30,-65.1,-65]

bbox=(df03['lons']>lonmin)&(df03['lons']<lonmax)&(df03['lats']>latmin)&(df03['lats']<latmax)
df_cut=df03[bbox]

## converting GPS time to UTC time
epoch=is2_f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df_cut['time']=Time(epoch+df_cut['dt'],format='gps').utc.datetime

## calculate along track distance relative to the beginning of the cut segment
# Subtract all along track x-values from first segment
df_cut['AT_dist']=df_cut.x-df_cut.x.values[0]

df_cut.head()

In [ ]:
plt.subplots(1,1,figsize=(10,4))
plt.scatter(df_cut.AT_dist.values[df_cut.conf<3],df_cut.heights.values[df_cut.conf<3],c='r',s=1)
plt.scatter(df_cut.AT_dist.values[df_cut.conf>2],df_cut.heights.values[df_cut.conf>2],c='g',s=1)
plt.xlabel('alongtrack distance (m)',fontsize=20)
plt.ylabel('heights',fontsize=20)
# plt.ylim(15,25);
plt.xlim(df_cut.AT_dist.values[0],df_cut.AT_dist.values[-1]);